In [40]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import gmaps
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

In [41]:
#Creating csv file path
output_data_file = "output_data/cities.csv"
cities_df = pd.read_csv(output_data_file,encoding="utf-8")
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [44]:
#Use the Lat and Lng as locations
locations = cities_df[['Lat','Lng']].astype(float)

cities_df = cities_df.dropna()

#Humidity as the weight
humidity = cities_df['Humidity'].astype(float)

In [45]:
#Configure gmaps
fig = gmaps.figure()

#Add Heatmap layer to map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
print(fig)

Figure(layout=FigureLayout(height='420px'))


In [46]:
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values
narrow_cities_df = cities_df.loc[(cities_df['Max Temp'] > 80) & (cities_df['Max Temp']<95) & (cities_df['Cloudiness']==0),:]
narrow_cities_df = narrow_cities_df.dropna(how='any')
narrow_cities_df.reset_index(inplace=True)
narrow_cities_df.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08
1,125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
2,142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
3,244,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17
4,316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17


In [47]:
#Store into variable named hotel_df
#Add a "Hotel Name" column to the DataFrame.
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap
#hotel_df = []

for x in range(len(narrow_cities_df)):
    lat = narrow_cities_df.loc[x]['Lat']
    lng = narrow_cities_df.loc[x]['Lng']
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": "hotel",
        "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_request = requests.get(base_url, params=params)
    requested = hotel_request.json()
    
    try:
        hotel_df.append(requested['results'][0]['name'])
        
    except:
        hotel_df.append("")
        
narrow_cities_df["Hotel Name"] = hotel_df
narrow_cities_df = narrow_cities_df.dropna(how='any')
narrow_cities_df.head()
        
        
            

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08,Colonelganj
1,125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,Cabo Frio
2,142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36,Sarai Meer
3,244,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17,Pasni
4,316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17,Gunjur


In [48]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrow_cities_df.iterrows()]
locations = narrow_cities_df[["Lat", "Lng"]]

In [51]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
plt.savefig('heatmap.png')
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>